In [3]:
import numpy as np
import pandas as pd

In [4]:
# Getting Data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [5]:
# Data Exploration / Analysis
train_df.info()
train_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 11 columns):
Pclass      668 non-null int64
Name        668 non-null object
Sex         668 non-null object
Age         536 non-null float64
SibSp       668 non-null int64
Parch       668 non-null int64
Ticket      668 non-null object
Fare        668 non-null float64
Cabin       154 non-null object
Embarked    667 non-null object
Survived    668 non-null int64
dtypes: float64(2), int64(4), object(5)
memory usage: 57.5+ KB


,Pclass,Age,SibSp,Parch,Fare,Survived
count,668.000000,536.000000,668.000000,668.000000,668.000000,668.000000
mean,2.296407,29.700560,0.528443,0.407186,32.064552,0.402695
std,0.831638,14.240257,1.080327,0.854695,45.320835,0.490808
min,1.000000,0.670000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,21.000000,0.000000,0.000000,7.925000,0.000000
50%,3.000000,29.000000,0.000000,0.000000,14.750000,0.000000
75%,3.000000,38.250000,1.000000,0.000000,31.275000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [7]:
# Missing Data
# Cabin
data = [train_df, test_df]

def getDeck(x) :
    return x[0:1]

for dataset in data :
    dataset['Cabin'].fillna('U0', inplace=True)
    dataset['Deck'] = dataset.Cabin.apply(getDeck)

train_df = train_df.drop(columns = ['Cabin'])
test_df = test_df.drop(columns = ['Cabin'])


In [8]:
# Age
survived_mean = train_df[train_df.Survived == 1].Age.mean()
not_survived_mean = train_df[train_df.Survived == 0].Age.mean()
test_mean = test_df.Age.mean()

train_df.update(train_df[train_df.Survived==1].Age.fillna(survived_mean))
train_df.update(train_df[train_df.Survived==0].Age.fillna(not_survived_mean))
test_df.Age.fillna(test_mean, inplace=True)

In [9]:
# Embarked
data = [train_df, test_df]
for dataset in data :
    dataset.Embarked.fillna('S', inplace=True)

In [10]:
# Drop Ticket and Name
train_df = train_df.drop(columns=['Ticket'])
test_df = test_df.drop(columns=['Ticket'])
train_df = train_df.drop(columns=['Name'])
test_df = test_df.drop(columns=['Name'])

In [11]:
# Getting Survived Column
y_df = train_df.Survived
train_df = train_df.drop(columns = ['Survived'])


In [12]:
# Categorical Data
def sex(x) :
    if x == 'male' :
        return 1
    else :
        return 0

def embarked(x) :
    if x == 'S' :
        return 0
    elif x == 'C' :
        return 1
    else:
        return 2

def deck(x) :
    dic = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'U':7}
    return dic[x]

def hello(x) :
    if x == 'T' :
        return 'A'
    else :
        return x

In [13]:
train_df.Deck = train_df.Deck.apply(hello)

for i in range(len(train_df.values)) :
    if train_df.iloc[i, 7] == 'T' :
        print(i)

In [14]:
data = [train_df, test_df]
for dataset in data :
    dataset['Sex'] = dataset['Sex'].apply(sex)
    dataset['Embarked'] = dataset['Embarked'].apply(embarked)
    dataset['Deck'] = dataset['Deck'].apply(deck)

In [15]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features=[1, 6, 7])
onehotencoder.fit(train_df)

C:\Users\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


OneHotEncoder(categorical_features=[1, 6, 7], categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [16]:
train_df = onehotencoder.transform(train_df).toarray()
test_df = onehotencoder.transform(test_df).toarray()

In [17]:
from sklearn.linear_model import LogisticRegression
algo = LogisticRegression()
algo.fit(train_df, y_df)
y_pred = algo.predict(test_df)


C:\Users\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
np.savetxt('sol.csv', y_pred, fmt="%d")